In [1]:
import yfinance as yf
import pymongo
import pandas as pd
import numpy as np
import plotly.express as px

In [2]:
client = pymongo.MongoClient("mongodb://localhost:27017/")

In [3]:
db = client["market_data"]

In [4]:
collection = db["stock_prices"]

In [5]:
symbols = ["AAPL", "MSFT", "GOOGL", "TSLA", "NVDA"]

In [10]:
def fetch_and_store(symbol, start_date, end_date):
    df = yf.download(symbol, start=start_date, end=end_date)
    df.columns = df.columns.get_level_values(0)
    records = [
        {
            "Ticker": symbol,
            "Date": row.Index.strftime("%Y-%m-%d"),
            "Open": row.Open,
            "High": row.High,
            "Low": row.Low,
            "Close": row.Close,
            "Volume": row.Volume
        }
        for row in df.itertuples()
    ]
    collection.insert_many(records)
    print(f"Inserted {len(records)} records for {symbol}")

In [11]:
for symbol in symbols:
    fetch_and_store(symbol, start_date="2020-01-01", end_date="2025-01-01")

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


Inserted 1258 records for AAPL
Inserted 1258 records for MSFT
Inserted 1258 records for GOOGL
Inserted 1258 records for TSLA


[*********************100%***********************]  1 of 1 completed

Inserted 1258 records for NVDA


In [ ]:
cursor = collection.find({}, {"_id": 0, "Ticker": 1, "Date": 1, "Close": 1})
df = pd.DataFrame(list(cursor))

In [9]:
grouped_df = df.groupby(["date", "symbol"]).last().reset_index()

In [10]:
df_pivot = grouped_df.pivot(index="date", columns="symbol", values="close")

In [11]:
df_pivot.columns = df_pivot.columns.get_level_values(0)

In [12]:
fig = px.line(df_pivot).update_layout(xaxis_title="Date", yaxis_title="Close Price USD", legend_title="Ticker")
fig.show()